# Using Slisemap on the Air Quality dataset

In this notebook we will demonstrate how to use Slisemap on the Air Quality dataset with a classification task.

In [ ]:
import sys

import numpy as np
import pandas as pd

from pathlib import Path
from urllib.request import urlretrieve

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sys.path.insert(0, "..")

from slisemap import Slisemap
from slisemap.local_models import logistic_regression, logistic_regression_loss

### Objectives

These are the objectives of this notebook:

- Demonstrate how to use a different type of white box model with Slisemap.
- Discuss the use of subsampling.
- Demonstrate how to tweak parameters.
- Investigate the solution and the data.
- Demonstrate how to add data not in the initial subsample.

## Slisemap Model Caching

To reduce execution times, we provide dumps of pretrained Slisemap models. If you want to train them yourself instead, please comment out the following lines and remove the `cache` directory in this `examples` folder.

In [ ]:
for cached in ["sm-02.v1.sm", "sm-02.v2.sm", "BZ-02.v3.npz"]:
    path = Path("cache") / cached
    path.parent.mkdir(exist_ok=True, parents=True)
    
    if not path.exists():
        urlretrieve(
            f"https://raw.githubusercontent.com/edahelsinki/slisemap/data/examples/cache/{cached}",
            path,
        )

## Data

The Air Quality dataset contains 7355 hourly instances of 12 different air quality measurements, one of which is used as a dependent variable (CO) and the others as covariates. Since this notebook demonstrates classification, we split the $y$-values into two clusters based on the median.

In [ ]:
# As in previous noteboook, load the dataset (download it if necessary).
path = Path("data") / "AQ_cleaned_version.csv"
path.parent.mkdir(exist_ok=True, parents=True)
if not path.exists():
    urlretrieve(
        "https://raw.githubusercontent.com/edahelsinki/drifter/master/TiittanenOHP2019-code/data/AQ_cleaned_version.csv",
        path,
    )
AQ = pd.read_csv(path)

In [ ]:
"""
The Air Quality dataset is cleaned and preprocessed as in:

    Oikarinen E, Tiittanen H, Henelius A, Puolamäki K (2021)
    Detecting virtual concept drift of regressors without ground truth values.
    Data Mining and Knowledge Discovery 35(3):726-747, DOI 10.1007/s10618-021-00739-7]
"""
columns = [
    "PT08.S1(CO)", "C6H6(GT)", "PT08.S2(NMHC)", "NOx(GT)", "PT08.S3(NOx)", "NO2(GT)",
    "PT08.S4(NO2)", "PT08.S5(O3)", "T", "RH", "AH",
]
names = [
    "CO(sensor)", "C6H6(GT)", "NMHC(sensor)", "NOx(GT)", "NOx(sensor)", "NO2(GT)",
    "NO2(sensor)", "O3(sensor)", "Temperature", "Relative hum.", "Absolute hum.",
]

# X contains the covariates, y is the target variable and names are column names.
X0 = AQ[columns].to_numpy()
y0 = AQ["CO(GT)"].to_numpy()

In [ ]:
pd.DataFrame(np.concatenate([y0.reshape((-1,1)), X0], axis=1), columns=["CO(GT)"]+names)

In [ ]:
# We will now split the target variables to two binary classes
# Straight through the middle based on target median
y = (y0 > np.median(y0)).astype(float)
y = np.stack((y, 1 - y), 1)
X = StandardScaler().fit_transform(X0)

### Black box model

Now, as in the previous notebook, we will use a black box model for the target value. For that we build a random forest classifier using `sklearn.ensemble.RandomForestClassifier`. Note that we use the predicted probabilities instead of the predicted labels!

In [ ]:
random_forest = RandomForestClassifier(random_state=42).fit(X, y)
y2 = random_forest.predict_proba(X)[0]

## Slisemap

Since we are now dealing with a classification task we replace the default white box model (linear regression) in Slisemap with logistic regression. Furthermore, 7355 data items is more than necessary for visualisation purposes, so we subsample 1500 data items. If results for all data items is needed we can add them later (using the structure from the 1500 samples for faster training).

In [ ]:
# Subsample the data
size = 1500
X, X_test, y2, y_test, X0, _ = train_test_split(
    X, y2, X0, train_size=size, test_size=size, stratify=y[:, 0], random_state=42
)

Now we create a Slisemap object (without an intercept) and optimise the solution. Since this example takes a while to optimise, we instead load a precalculated Slisemap object from disk.

In [ ]:
path = Path("cache") / "sm-02.v1.sm"
path.parent.mkdir(exist_ok=True, parents=True)

if path.exists():
    sm = Slisemap.load(path)
else:
    # Create a Slisemap object with lasso regularisation
    sm = Slisemap(
        X,
        y2,
        lasso=0.0001,
        radius=3.5,
        intercept=False,
        local_model=logistic_regression,
        local_loss=logistic_regression_loss,
        coefficients=X.shape[1],
        random_state=42,
    )
    # Optimise the solution
    %time sm.optimise()

    sm.cpu()
    sm.save(path)

print(f"Loss: {sm.value()}")

### Tweaking

If we plot the solution we see that the linear models are not very sparse even though we used lasso/L1 regularisation:

In [ ]:
sm.plot(title="Slisemap with Air Quality data", variables=names)

This is beacuse we used quite weak regularisation. We can increase the regularisation strength without creating a new `Slisemap` object (remember to re-optimise):

In [ ]:
path = Path("cache") / "sm-02.v2.sm"
path.parent.mkdir(exist_ok=True, parents=True)

if Path(path).exists():
    sm = Slisemap.load(path)
else:
    sm.lasso = 0.005
    %time sm.optimise()

    sm.cpu()
    sm.save(path)

In [ ]:
sm.plot(title="Slisemap with Air Quality data", variables=names)

Now the local models are more sparse (more coefficients are zero). This makes them easier to interpret. Additionally, the clusters in the embedding are more distinct, due to the larger differences in the local models. 

### Visualisations

Next we will add some jitter and plot the local models in a bar plot:

In [ ]:
clusters = 5
bars = 7
sm.plot(
    title="Slisemap with local model clusters",
    clusters=clusters,
    bars=bars,
    jitter=0.1,
    variables=names,
)

This makes it easier to see which clusters and local models are connected, and to compare the magnitudes of the coefficients of the local models.

Next we plot the distribution of attribute values for the clusters (note the use of unscaled data):

In [ ]:
sm.plot_dist(
    X=X0,
    title="Distributions of the variable values",
    clusters=clusters,
    variables=names,
    targets="CO(GT)",
    height=3,
)

Here we see that the clusters do not only have different local models, there are also differences in the distributions of the variables.

## Unseen test data

Initially we subsampled the data into the training set consisting of 1500 data items and the testing set of 1500. We will now examine how data items not used for training the Slisemap solution could be added to the embedding. First we take 7 random data items and show what their fidelities would be if we were to copy the embedding and local model from one training data item (here we plot every possible choice).

In [ ]:
np.random.seed(42)
sel = np.random.randint(0, X_test.shape[0], 7)
sm.plot_position(
    X_test[sel],
    y_test[sel],
    title="Projection of unseen datapoints",
    height=3,
    jitter=0.1,
)

Here we see that the local models in the different clusters are not suitable for every unseen data item.

Next we will optimise new embeddings and new local models for all the unseen data, using the structure from the training data as a guide:

In [ ]:
path = Path("cache") / "BZ-02.v3.npz"
path.parent.mkdir(exist_ok=True, parents=True)

if Path(path).exists():
    BZ = np.load(path)
    B_test = BZ["B_test"]
    Z_test = BZ["Z_test"]
else:
    %time B_test, Z_test = sm.fit_new(X_test, y_test)

    np.savez_compressed(path, B_test=B_test, Z_test=Z_test)

Note that this is much faster than training the solution from scratch.

Finally, lets plot the embeddings of the new and old data in the same plot:

In [ ]:
Zcon = np.concatenate((sm.get_Z(), Z_test), 0)
Zcon = Zcon @ np.linalg.svd(Zcon)[2].T # Rotate to match previous plots
sm.plot(
    B=np.concatenate((sm.get_B(), B_test), 0),
    Z=Zcon,
    title="test",
    clusters=["old"]*sm.n + ["new"]*B_test.shape[0],
    jitter=0.1,
)

As we can see the structure of the unseen data matches that of the training data.

## Conclusions

In this notebook we have looked at a simple classifier use case for Slisemap, and demonstrated the various plots that can be used to to visualise, explore, and investigate the solution and the data.